Regressions between ERA5 (dif. variables) and Cloud Index (CI) in Corrientes Argentina

Las anomalias de CI se generaron asi:

* Calculo el CI mensuales para los meses con menos del 20% de datos faltantes
* Calculo las anomalias de CI mensuales (le resto a las medias mensuales la CI climatologica de cada mes calculada sobre el periodo comun con el satelite: fecha_inicio_climatologia_str = "1983-12-15"
fecha_final_climatologia_str = "2016-11-15")
* Todo se guarda en csvs: cada columna es una estacion. En: "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2021/nubosidad/paper"
TCC_monthly_CI_SMN_anomaly.csv

ERA5 is for southamerica: 
        lat_min = -60
        lat_max = 15
        lon_min = -90
        lon_max = -30

In [8]:
import os
import pandas as pd 
import numpy as np

import iris
from iris.analysis import MEAN
from iris.analysis.stats import pearsonr
import iris.coord_categorisation

import scipy
from scipy.stats import theilslopes #theil-sen trends

import matplotlib.pyplot as plt

In [3]:
# directories
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

#si estoy trabajando desde el servidor vegeta
ruta_a_archivos_ERA5_mensual = "/pikachu/datos/nadia.testani/Doctorado/datos/ERA5/mon/SAm" #ruta donde estan los archivos con datos de ERA5
ruta_a_archivos_CI = "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2021/nubosidad/paper"
ruta_a_resultados = "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2024" #ruta donde se guardan los resultados del analisis de nubosidad

Current working directory: /home/nadia.testani/Doctorado/DoctoradoCodigos/codigos2024


In [18]:
#Cloud Index

CI_anomaly = pd.read_csv(f"{ruta_a_archivos_CI}/TCC_monthly_CI_SMN_anomaly.csv")
#from 1961-01-01 until 2021-11-01

#set dates as index
CI_anomaly = CI_anomaly.set_index(CI_anomaly["Unnamed: 0"]) 
CI_anomaly = CI_anomaly.drop(["Unnamed: 0"], axis = 1)
CI_anomaly.index = pd.DatetimeIndex(CI_anomaly.index)

#filter theil-sen trend
def calculate_theilsen_slope(y):
    slope, intercept, _, _ = theilslopes(y, range(len(y)), alpha=0.95)
    return slope, intercept

CI_anomaly_no_trends = CI_anomaly.copy()
for column in CI_anomaly_no_trends: #iterate throw every station
    serie = CI_anomaly_no_trends[column]
    valores = serie.values[np.isfinite(serie.values)]
    trend = calculate_theilsen_slope(valores)[0] * np.arange(len(serie)) + calculate_theilsen_slope(valores)[1]
    CI_anomaly_no_trends[column] = serie - trend

#regional mean value
CI_anomaly_no_trends_regional_mean = CI_anomaly_no_trends.mean(axis=1)

In [22]:
#ERA5
ERA5_netcdf = iris.load_cube(f"{ruta_a_archivos_ERA5_mensual}/ERA5_zg_SA.nc")
ERA5_netcdf

<iris 'Cube' of Total precipitation / (m) (time: 756; latitude: 201; longitude: 201)>